In [2]:
pip install pandas_datareader

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import json
import os
import time
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import pandas_datareader.data as web


In [5]:
class SP500DataCollector:
    def __init__(self):
        # 加载配置
        with open('config.json', 'r') as f:
            self.config = json.load(f)
        
        # 创建数据存储文件夹
        for folder in ['stock_data_folder', 'factor_data_folder']:
            folder_path = self.config['output_settings'][folder]
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

    def get_sp500_list(self):
        """获取S&P500成分股列表"""
        url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', {'class': 'wikitable'})
        
        df = pd.DataFrame(columns=['Symbol', 'Company', 'Sector'])
        
        for row in table.findAll('tr')[1:]:
            cols = row.findAll('td')
            if len(cols) >= 3:
                symbol = cols[0].text.strip()
                company = cols[1].text.strip()
                sector = cols[3].text.strip()
                df = pd.concat([df, pd.DataFrame({
                    'Symbol': [symbol],
                    'Company': [company],
                    'Sector': [sector]
                })], ignore_index=True)
        
        # 添加验证
        if len(df) != 500:
            print(f"警告：获取到的SP500公司数量为 {len(df)}，而不是500家")
        
        # 保存SP500列表
        df.to_csv(self.config['output_settings']['sp500_list_file'], index=False)
        return df

    def get_stock_data(self, symbol):
        """获取单个股票的历史数据"""
        settings = self.config['data_settings']
        stock = yf.Ticker(symbol)
        
        try:
            data = stock.history(
                start=settings['start_date'],
                end=settings['end_date']
            )
            
            # 只保留配置文件中指定的指标
            data = data[settings['indicators']]
            
            # 保存数据
            output_path = os.path.join(
                self.config['output_settings']['stock_data_folder'],
                f"{symbol}.csv"
            )
            data.to_csv(output_path)
            return True
        except Exception as e:
            print(f"获取 {symbol} 数据时出错: {str(e)}")
            return False

    def calculate_technical_factors(self, data):
        """计算技术面因子"""
        tech_settings = self.config['factor_settings']['technical_factors']
        
        # 计算动量因子
        for period in tech_settings['momentum_periods']:
            data[f'momentum_{period}d'] = data['Close'].pct_change(period)
        
        # 计算RSI
        def calculate_rsi(prices, period=14):
            delta = prices.diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
            rs = gain / loss
            return 100 - (100 / (1 + rs))
        
        data['RSI'] = calculate_rsi(data['Close'], tech_settings['rsi_period'])
        
        # 计算MACD
        macd_params = tech_settings['macd_params']
        exp1 = data['Close'].ewm(span=macd_params['fast_period']).mean()
        exp2 = data['Close'].ewm(span=macd_params['slow_period']).mean()
        data['MACD'] = exp1 - exp2
        data['Signal_Line'] = data['MACD'].ewm(span=macd_params['signal_period']).mean()
        data['MACD_Histogram'] = data['MACD'] - data['Signal_Line']
        
        return data

    def get_fundamental_factors(self, symbol):
        """获取基本面因子"""
        stock = yf.Ticker(symbol)
        fund_metrics = {}
        
        # 获取基本面指标
        try:
            info = stock.info
            metrics = self.config['factor_settings']['fundamental_factors']['metrics']
            
            for metric in metrics:
                fund_metrics[metric] = info.get(metric, None)
                
            # 获取财务报表数据
            financials = stock.financials
            if not financials.empty:
                # 计算净利润增长率
                net_income = financials.loc['Net Income']
                fund_metrics['net_income_growth'] = (
                    (net_income.iloc[0] - net_income.iloc[1]) / 
                    abs(net_income.iloc[1]) * 100 if len(net_income) >= 2 else None
                )
        except Exception as e:
            print(f"获取 {symbol} 基本面数据时出错: {str(e)}")
        
        return fund_metrics

    def get_market_factors(self):
        """获取Fama-French三因子数据"""
        try:
            ff_factors = web.DataReader(
                'F-F_Research_Data_Factors_daily',
                'famafrench',
                start=self.config['data_settings']['start_date'],
                end=self.config['data_settings']['end_date']
            )[0]
            
            # 将百分比转换为小数
            ff_factors = ff_factors / 100
            
            # 保存因子数据
            output_path = os.path.join(
                self.config['output_settings']['factor_data_folder'],
                'ff_factors.csv'
            )
            ff_factors.to_csv(output_path)
            return ff_factors
        except Exception as e:
            print(f"获取Fama-French因子数据时出错: {str(e)}")
            return None

    def process_stock_data(self, symbol):
        """处理单个股票的所有数据"""
        settings = self.config['data_settings']
        stock = yf.Ticker(symbol)
        
        try:
            # 获取价格数据
            data = stock.history(
                start=settings['start_date'],
                end=settings['end_date']
            )
            
            # 计算技术面因子
            if self.config['factor_settings']['technical_factors']['enabled']:
                data = self.calculate_technical_factors(data)
            
            # 获取基本面因子
            if self.config['factor_settings']['fundamental_factors']['enabled']:
                fund_factors = self.get_fundamental_factors(symbol)
                # 将基本面因子添加为常量列
                for factor, value in fund_factors.items():
                    data[factor] = value
            
            # 保存数据
            output_path = os.path.join(
                self.config['output_settings']['stock_data_folder'],
                f"{symbol}_full.csv"
            )
            data.to_csv(output_path)
            return True
        except Exception as e:
            print(f"处理 {symbol} 数据时出错: {str(e)}")
            return False

    def collect_all_data(self):
        """收集所有数据"""
        # 获取市场因子
        if self.config['factor_settings']['market_factors']['enabled']:
            print("正在获取Fama-French因子数据...")
            self.get_market_factors()
        
        # 获取SP500列表
        sp500_df = self.get_sp500_list()
        print(f"获取到 {len(sp500_df)} 个SP500成分股")

        # 处理每个股票的数据
        for index, row in sp500_df.iterrows():
            symbol = row['Symbol']
            print(f"正在处理 {symbol} 的数据...")
            
            success = False
            retries = self.config['api_settings']['retry_times']
            
            while retries > 0 and not success:
                success = self.process_stock_data(symbol)
                if not success:
                    retries -= 1
                    time.sleep(self.config['api_settings']['sleep_between_calls'])
            
            if success:
                print(f"{symbol} 数据处理成功")
            else:
                print(f"{symbol} 数据处理失败")

In [6]:
def main():
    collector = SP500DataCollector()
    collector.collect_all_data()

if __name__ == "__main__":
    main() 

正在获取Fama-French因子数据...


/var/folders/cw/yxxjm9s13cz54l3wm4btmtnc0000gn/T/ipykernel_35938/2414291931.py:125: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_factors = web.DataReader(


警告：获取到的SP500公司数量为 503，而不是500家
获取到 503 个SP500成分股
正在处理 MMM 的数据...
MMM 数据处理成功
正在处理 AOS 的数据...
AOS 数据处理成功
正在处理 ABT 的数据...
ABT 数据处理成功
正在处理 ABBV 的数据...
ABBV 数据处理成功
正在处理 ACN 的数据...
ACN 数据处理成功
正在处理 ADBE 的数据...
ADBE 数据处理成功
正在处理 AMD 的数据...
AMD 数据处理成功
正在处理 AES 的数据...
AES 数据处理成功
正在处理 AFL 的数据...
AFL 数据处理成功
正在处理 A 的数据...
A 数据处理成功
正在处理 APD 的数据...
APD 数据处理成功
正在处理 ABNB 的数据...
ABNB 数据处理成功
正在处理 AKAM 的数据...
AKAM 数据处理成功
正在处理 ALB 的数据...
ALB 数据处理成功
正在处理 ARE 的数据...
ARE 数据处理成功
正在处理 ALGN 的数据...
ALGN 数据处理成功
正在处理 ALLE 的数据...
ALLE 数据处理成功
正在处理 LNT 的数据...
LNT 数据处理成功
正在处理 ALL 的数据...
ALL 数据处理成功
正在处理 GOOGL 的数据...
GOOGL 数据处理成功
正在处理 GOOG 的数据...
GOOG 数据处理成功
正在处理 MO 的数据...
MO 数据处理成功
正在处理 AMZN 的数据...
AMZN 数据处理成功
正在处理 AMCR 的数据...
AMCR 数据处理成功
正在处理 AEE 的数据...
AEE 数据处理成功
正在处理 AEP 的数据...
AEP 数据处理成功
正在处理 AXP 的数据...
AXP 数据处理成功
正在处理 AIG 的数据...
AIG 数据处理成功
正在处理 AMT 的数据...
AMT 数据处理成功
正在处理 AWK 的数据...
AWK 数据处理成功
正在处理 AMP 的数据...
AMP 数据处理成功
正在处理 AME 的数据...
AME 数据处理成功
正在处理 AMGN 的数据...
AMGN 数据处理成功
正在处理 APH 的数据...
APH 数据处理成功
正在处理 ADI 的数据...


$BRK.B: possibly delisted; no timezone found


BRK.B 数据处理成功
正在处理 BBY 的数据...
BBY 数据处理成功
正在处理 TECH 的数据...
TECH 数据处理成功
正在处理 BIIB 的数据...
BIIB 数据处理成功
正在处理 BLK 的数据...
BLK 数据处理成功
正在处理 BX 的数据...
BX 数据处理成功
正在处理 BK 的数据...
BK 数据处理成功
正在处理 BA 的数据...
BA 数据处理成功
正在处理 BKNG 的数据...
BKNG 数据处理成功
正在处理 BWA 的数据...
BWA 数据处理成功
正在处理 BSX 的数据...
BSX 数据处理成功
正在处理 BMY 的数据...
BMY 数据处理成功
正在处理 AVGO 的数据...
AVGO 数据处理成功
正在处理 BR 的数据...
BR 数据处理成功
正在处理 BRO 的数据...


...

GEV 数据处理成功
正在处理 GEN 的数据...
GEN 数据处理成功
正在处理 GNRC 的数据...
GNRC 数据处理成功
正在处理 GD 的数据...
GD 数据处理成功
正在处理 GIS 的数据...
GIS 数据处理成功
正在处理 GM 的数据...
GM 数据处理成功
正在处理 GPC 的数据...
GPC 数据处理成功
正在处理 GILD 的数据...
GILD 数据处理成功
正在处理 GPN 的数据...
GPN 数据处理成功
正在处理 GL 的数据...
GL 数据处理成功
正在处理 GDDY 的数据...
GDDY 数据处理成功
正在处理 GS 的数据...
GS 数据处理成功
正在处理 HAL 的数据...
HAL 数据处理成功
正在处理 HIG 的数据...
HIG 数据处理成功
正在处理 HAS 的数据...
HAS 数据处理成功
正在处理 HCA 的数据...
HCA 数据处理成功
正在处理 DOC 的数据...
DOC 数据处理成功
正在处理 HSIC 的数据...
HSIC 数据处理成功
正在处理 HSY 的数据...
HSY 数据处理成功
正在处理 HES 的数据...
HES 数据处理成功
正在处理 HPE 的数据...
HPE 数据处理成功
正在处理 HLT 的数据...
HLT 数据处理成功
正在处理 HOLX 的数据...
HOLX 数据处理成功
正在处理 HD 的数据...
HD 数据处理成功
正在处理 HON 的数据...
HON 数据处理成功
正在处理 HRL 的数据...
HRL 数据处理成功
正在处理 HST 的数据...
HST 数据处理成功
正在处理 HWM 的数据...
HWM 数据处理成功
正在处理 HPQ 的数据...
HPQ 数据处理成功
正在处理 HUBB 的数据...
HUBB 数据处理成功
正在处理 HUM 的数据...
HUM 数据处理成功
正在处理 HBAN 的数据...
HBAN 数据处理成功
正在处理 HII 的数据...
HII 数据处理成功
正在处理 IBM 的数据...
IBM 数据处理成功
正在处理 IEX 的数据...
IEX 数据处理成功
正在处理 IDXX 的数据...
IDXX 数据处理成功
正在处理 ITW 的数据...
ITW 数据处理成功
正在处理 INCY 的

$SW: possibly delisted; no price data found  (1d 2022-01-01 -> 2024-03-20) (Yahoo error = "Data doesn't exist for startDate = 1641013200, endDate = 1710907200")


SW 数据处理成功
正在处理 SNA 的数据...
SNA 数据处理成功
正在处理 SOLV 的数据...


$SOLV: possibly delisted; no price data found  (1d 2022-01-01 -> 2024-03-20) (Yahoo error = "Data doesn't exist for startDate = 1641013200, endDate = 1710907200")


SOLV 数据处理成功
正在处理 SO 的数据...
SO 数据处理成功
正在处理 LUV 的数据...
LUV 数据处理成功
正在处理 SWK 的数据...
SWK 数据处理成功
正在处理 SBUX 的数据...
SBUX 数据处理成功
正在处理 STT 的数据...
STT 数据处理成功
正在处理 STLD 的数据...
STLD 数据处理成功
正在处理 STE 的数据...
STE 数据处理成功
正在处理 SYK 的数据...
SYK 数据处理成功
正在处理 SMCI 的数据...
SMCI 数据处理成功
正在处理 SYF 的数据...
SYF 数据处理成功
正在处理 SNPS 的数据...
SNPS 数据处理成功
正在处理 SYY 的数据...
SYY 数据处理成功
正在处理 TMUS 的数据...
TMUS 数据处理成功
正在处理 TROW 的数据...
TROW 数据处理成功
正在处理 TTWO 的数据...
TTWO 数据处理成功
正在处理 TPR 的数据...
TPR 数据处理成功
正在处理 TRGP 的数据...
TRGP 数据处理成功
正在处理 TGT 的数据...
TGT 数据处理成功
正在处理 TEL 的数据...
TEL 数据处理成功
正在处理 TDY 的数据...
TDY 数据处理成功
正在处理 TFX 的数据...
TFX 数据处理成功
正在处理 TER 的数据...
TER 数据处理成功
正在处理 TSLA 的数据...
TSLA 数据处理成功
正在处理 TXN 的数据...
TXN 数据处理成功
正在处理 TPL 的数据...
TPL 数据处理成功
正在处理 TXT 的数据...
TXT 数据处理成功
正在处理 TMO 的数据...
TMO 数据处理成功
正在处理 TJX 的数据...
TJX 数据处理成功
正在处理 TSCO 的数据...
TSCO 数据处理成功
正在处理 TT 的数据...
TT 数据处理成功
正在处理 TDG 的数据...
TDG 数据处理成功
正在处理 TRV 的数据...
TRV 数据处理成功
正在处理 TRMB 的数据...
TRMB 数据处理成功
正在处理 TFC 的数据...
TFC 数据处理成功
正在处理 TYL 的数据...
TYL 数据处理成功
正在处理 TSN 的数据...
TSN 数据处理成